# Result plots

In [11]:
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [13]:
def create_3d_numpy_array_with_folder_mapping(base_directory):
    # Define configurations, learning methods, and folders
    configurations = ['multi-step-ahead-direct', 'multi-step-ahead-iterative', 'single-step-ahead']
    learning_methods = ['LSTM', 'RandomForest']

    # Scan the base directory for folders (Z-axis)
    folders = sorted([f for f in os.listdir(base_directory) if os.path.isdir(os.path.join(base_directory, f))])

    # Initialize the 3D array (Python list first)
    model_3d_array = [[[None for _ in configurations] for _ in learning_methods] for _ in folders]
    folder_mapping = {}

    # Function to load JSON file
    def load_json_file(filepath):
        with open(filepath, 'r') as file:
            return json.load(file)

    # Iterate over the folders (Z-axis)
    for z_idx, folder in enumerate(folders):
        folder_path = os.path.join(base_directory, folder)

        # Map folder name to Z-axis index
        folder_mapping[folder] = z_idx

        # Iterate over the files in each folder
        for filename in os.listdir(folder_path):
            if filename.endswith(".json"):
                # Determine the model type (LSTM or RandomForest) from the filename
                if "LSTM" in filename:
                    y_idx = 0  # LSTM corresponds to Y-axis index 0
                elif "RandomForest" in filename:
                    y_idx = 1  # RandomForest corresponds to Y-axis index 1
                else:
                    continue  # Skip files that don't match the model types

                # Determine the configuration type (X-axis index)
                if "multi-step-ahead-direct" in filename:
                    x_idx = 0
                elif "multi-step-ahead-iterative" in filename:
                    x_idx = 1
                elif "single-step-ahead" in filename:
                    x_idx = 2
                else:
                    continue  # Skip files that don't match the configuration types

                # Load the JSON data and store it in the 3D array
                filepath = os.path.join(folder_path, filename)
                model_3d_array[z_idx][y_idx][x_idx] = load_json_file(filepath)

    # Convert the Python list (3D array) to a NumPy array with dtype=object
    model_3d_numpy_array = np.array(model_3d_array, dtype=object)

    return model_3d_numpy_array, folder_mapping

# Usage example
base_directory = os.getcwd() + "/json"  # Set this to the path where your folders (e.g., balanced_10, hybrid_1, etc.) are located
model_3d_numpy_array, folder_mapping = create_3d_numpy_array_with_folder_mapping(base_directory)

# Print the NumPy array structure and folder mapping to verify
print("3D Model NumPy Array:", model_3d_numpy_array.shape)
for key, value in folder_mapping.items():
    print(key, ":", value)


3D Model NumPy Array: (18, 2, 3)
balanced_10 : 0
balanced_100 : 1
balanced_150 : 2
balanced_50 : 3
hybrid_1 : 4
hybrid_4 : 5
hybrid_9 : 6
split_12_12 : 7
split_18_6 : 8
split_6_6 : 9
split_6_9 : 10
split_9_3 : 11
split_9_6 : 12
unbalanced_10 : 13
unbalanced_100 : 14
unbalanced_25 : 15
unbalanced_5 : 16
unbalanced_50 : 17
